In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
import yfinance as yf
import requests
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
api_key = os.getenv("API_KEY")

In [4]:
warnings.filterwarnings('ignore', category=FutureWarning)


In [40]:

# Carregando os dados dos arquivos CSV
rf_file_path = 'data/rf_br_10y.csv'
cds_file_path = 'data/br_cds_10y.csv'

# Lendo os arquivos CSV
riskfree = pd.read_csv(rf_file_path, delimiter=";")
cdsbrasil = pd.read_csv(cds_file_path, delimiter=";")

# Extraindo as taxas de título livre de risco para anos específicos
rf_data = {
    "Ano": [2018, 2019, 2020, 2021, 2022],
    "Taxa Livre de Risco": [
        riskfree.loc[1184, 'Último'],
        riskfree.loc[932, 'Último'],
        riskfree.loc[680, 'Último'],
        riskfree.loc[426, 'Último'],
        riskfree.loc[0, 'Último']
    ]
}

# Extraindo os valores para o CDS
cds_data = {
    "Ano": [2018, 2019, 2020, 2021, 2022],
    "Taxa CDS": [
        cdsbrasil.loc[1230, "Último"] / 100,
        cdsbrasil.loc[969, "Último"] / 100,
        cdsbrasil.loc[708, "Último"] / 100,
        cdsbrasil.loc[447, "Último"] / 100,
        cdsbrasil.loc[0, "Último"] / 100
    ]
}

# Convertendo em DataFrames
rf_df = pd.DataFrame(rf_data)
cds_df = pd.DataFrame(cds_data)

# Combinando os dois DataFrames
combined_df = pd.merge(rf_df, cds_df, on="Ano")

riskfree.head()
combined_df


,Ano,Tava Livre de Risco,Taxa CDS
0,2018,10.080,2.42980
1,2019,9.250,2.70050
2,2020,6.725,1.75080
3,2021,6.878,2.08161
4,2022,11.800,3.42900


In [32]:
# interest_rate_df = pd.read_csv('interest_rates.csv')

# plt.figure(figsize=(14, 6))

# sns.lineplot(data=interest_rate_df, x='Term', y='Rates 2023', marker='o', color='#1f77b4', 
#              label='United States Treasury Constant Maturity - 2023')
# sns.lineplot(data=interest_rate_df, x='Term', y='Rates 2022', marker='D', color='#ff7f0e',
#              label='United States Treasury Constant Maturity - 2022')


# plt.title('Risk-Free Rate Term Structure')
# plt.xlabel('Term (Years)')
# plt.ylabel('Rate (%)')
# plt.grid(True)

# plt.show()


# Veja a partir daqui

## risk-free rate 10-Year American Treasury Yield

In [9]:
def fetch_treasury_yield(api_key):
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TREASURY_YIELD",
        "interval": "monthly",
        "maturity": "10year",
        "apikey": api_key,
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        latest_date = max(data['data'], key=lambda x: x['date'])
        latest_yield = latest_date['value']
        return f"{float(latest_yield)}%"
    else:
        print(f"Não foi possível fetch: {response.status_code}")
        return None



risk_free_rate = fetch_treasury_yield(api_key)

if risk_free_rate:
    print(f"Taxa livre de risco (10-Year American Treasury Yield): {risk_free_rate}")

Taxa livre de risco (10-Year American Treasury Yield): 4.5%


## Beta

In [7]:
# Vibra Energia
stock_ticker = 'VBBR3.SA'  
# S&P 500
market_index_ticker = '^GSPC'  
start_date = '2018-01-01'
end_date = '2023-01-01'

stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)

# Cálculo de daily returns
stock_returns = stock_data['Adj Close'].pct_change().dropna()
market_returns = market_data['Adj Close'].pct_change().dropna()


aligned_returns = pd.concat([stock_returns, market_returns], axis=1).dropna()
aligned_returns.columns = ['Stock Returns', 'Market Returns']

# Regressão para o cálculo do Beta
X = sm.add_constant(aligned_returns['Market Returns'])  
# Ordinary Least Squares regression (OLS) 
model = sm.OLS(aligned_returns['Stock Returns'], X).fit()
beta = model.params['Market Returns']

# Display the beta
print(f"Beta: {beta}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Beta: 0.7411625858272158


In [46]:
annual_beta = aligned_returns['Beta'].resample('Y').mean()
annual_beta.name = 'Beta Médio Por Ano'

annual_beta_df = annual_beta.to_frame()


display(annual_beta_df)


,Beta Médio Por Ano
Date,
2018-12-31,0.539033
2019-12-31,0.141874
2020-12-31,0.823014
2021-12-31,0.460761
2022-12-31,0.507617


In [4]:
# # Data for each component
# ebitda = np.array([1.859, 1.839, 5.222, 3.833, 3.649, 5.894, 7.316, 7.784, 8.291, 8.827])
# taxes = np.array([-1.616, -1.092, -0.925, -0.677, -0.393, -1.714, -2.093, -2.141, -2.197, -2.256])
# change_in_working_capital = np.array([-3.899, 0.357, 0.366, -3.021, 0.059, -3.511, -1.507, -0.605, -0.647, -0.682])
# capex = np.array([-0.196, -0.373, -0.581, -2.415, -2.011, -2.669, -3.091, -3.256, -3.432, -3.617])

# # Calculate the Cash Flow (FCFF) for each year
# cash_flow = ebitda + taxes + change_in_working_capital + capex

# # Discount factors and periods of discount (assuming the periods start from 0.5 to 4.5 with 1 year increments)
# discount_periods = np.array([0.5, 1.5, 2.5, 3.5, 4.5])
# discount_rate = 0.104  # 10.4% discount rate
# # Discount factors calculated using the formula: 1 / (1 + r)^t
# discount_factors = 1 / (1 + discount_rate) ** discount_periods

# # Calculate the present value of the cash flows for the years where discount factors are given
# present_values = cash_flow[5:10] * discount_factors

# # Create a DataFrame to display the data neatly
# fcff_df = pd.DataFrame({
#     'Ano': np.arange(2018, 2028),
#     'EBITDA': ebitda,
#     'IRPJ/CSLL': taxes,
#     'Variação do capital de giro': change_in_working_capital,
#     'CAPEX': capex,
#     'Fluxo de Caixa': cash_flow,
#     'Fator de desconto': np.append(np.full(5, np.nan), discount_factors),  
#     # 'Present Value': np.append(np.full(5, np.nan), present_values)  
# })

# fcff_df
